In [1]:
import vaex
import geopandas as gpd
import pandas as pd

/home/fernando/miniconda3/envs/dash/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
ano = 2017

In [3]:
gdf_distritos = gpd.read_file('data/SIRGAS_GPKG_distrito.gpkg')

In [6]:
agg_atributos = {
        'Quantidade de Unidades':'sum',
        'Quantidade de Unidades Condominiais':'sum',
        'Tamanho Médio da Unidade Condominial':'mean',
        'Tamanho médio dos Terrenos':'mean',
        'Área Total dos terrenos-lotes':'sum',
        'Área Total Ocupada':'sum',
        'Área Total Construída':'sum',
        'Valor Total dos Terrenos':'sum',
        'Valor Total das Construções':'sum',
        'CA médio':'mean',
        'TO médio':'mean',
        'CA médio em lotes condominiais':'mean',
        'TO médio em lotes condominiais':'mean',
        'CA médio em lotes não condominiais':'mean',
        'TO médio em lotes não condominiais':'mean',
        'Comprimento Médio da Testada':'mean',
        'Número médio de Pavimentos':'mean',
        'Fator de obsolecência médio':'mean',
        'Residencial vertical Baixo (m2)':'sum',
        'Residencial vertical Médio (m2)':'sum',
        'Residencial vertical Alto (m2)':'sum',
        'Residencial horizontal Baixo (m2)':'sum',
        'Residencial horizontal Médio (m2)':'sum',
        'Residencial horizontal Alto (m2)':'sum',
        'Comercial vertical Baixo (m2)':'sum',
        'Comercial vertical Médio (m2)':'sum',
        'Comercial vertical Alto (m2)':'sum',
        'Comercial horizontal Baixo (m2)':'sum',
        'Comercial horizontal Alto (m2)':'sum',
        'Comercial horizontal Médio (m2)':'sum',
        'Terreno (m2)':'sum',
        'Outros Usos (m2)':'sum',
        'Residencial vertical Baixo (qt)':'sum',
        'Residencial vertical Médio (qt)':'sum',
        'Residencial vertical Alto (qt)':'sum',
        'Residencial horizontal Baixo (qt)':'sum',
        'Residencial horizontal Médio (qt)':'sum',
        'Residencial horizontal Alto (qt)':'sum',
        'Comercial vertical Baixo (qt)':'sum',
        'Comercial vertical Médio (qt)':'sum',
        'Comercial vertical Alto (qt)':'sum',
        'Comercial horizontal Baixo (qt)':'sum',
        'Comercial horizontal Alto (qt)':'sum',
        'Comercial horizontal Médio (qt)':'sum',
        'Terreno (qt)':'sum',
        'Outros Usos (qt)':'sum'
}

In [7]:
for _, distrito in gdf_distritos.iterrows():

    # Abrindo Arquivo de lotes
    path = f'lotes_agregados_por_ano/{ano}/SIRGAS_SHP_LOTES_{distrito.ds_codigo.rjust(2, "0")}_{distrito.ds_nome.replace(" ", "_")}_IPTU_{ano}.gpkg'
    gdf_lote = gpd.read_file(path).drop_duplicates(subset=['sqlc']).set_index('sqlc')
    #gdf_lote.to_crs(epsg=4674, inplace=True)
    gdf_lote = gdf_lote[gdf_lote.is_valid]

    df_iptu = vaex.open(f'data/por_distritos/IPTU-1995-2022-agrupados-por-sqlc-{distrito.ds_codigo}-{distrito.ds_nome.replace(" ", "-").lower()}.hdf5').to_pandas_df().set_index('sqlc')

    df_iptu = df_iptu[df_iptu.ano == ano]#[[atributo]]
    lotes_existentes = gdf_lote.join(df_iptu, how='inner')
    lotes_sg = df_iptu.join(gdf_lote, how='left').sq.isna()
    df_lotes_sg = df_iptu[lotes_sg].reset_index()
    df_lotes_sg.sqlc = df_lotes_sg.sqlc.str[:6] + '000000'
    df_lotes_sg_group = df_lotes_sg.groupby('sqlc').agg(agg_atributos)
    lotes_agregados = gdf_lote.join(df_lotes_sg_group, how='inner')
    lotes = pd.concat([lotes_existentes, lotes_agregados])

    #salvar-lotes
    
    break

KeyError: "Column(s) ['Comercial horizontal Alto (m2)', 'Comercial horizontal Alto (qt)', 'Comercial horizontal Baixo (m2)', 'Comercial horizontal Baixo (qt)', 'Comercial horizontal Médio (m2)', 'Comercial horizontal Médio (qt)', 'Comercial vertical Alto (m2)', 'Comercial vertical Alto (qt)', 'Comercial vertical Baixo (m2)', 'Comercial vertical Baixo (qt)', 'Comercial vertical Médio (m2)', 'Comercial vertical Médio (qt)', 'Outros Usos (m2)', 'Outros Usos (qt)', 'Residencial horizontal Alto (m2)', 'Residencial horizontal Alto (qt)', 'Residencial horizontal Baixo (m2)', 'Residencial horizontal Baixo (qt)', 'Residencial horizontal Médio (m2)', 'Residencial horizontal Médio (qt)', 'Residencial vertical Alto (m2)', 'Residencial vertical Alto (qt)', 'Residencial vertical Baixo (m2)', 'Residencial vertical Baixo (qt)', 'Residencial vertical Médio (m2)', 'Residencial vertical Médio (qt)', 'Terreno (m2)', 'Terreno (qt)'] do not exist"

In [10]:
df_iptu.columns

Index(['ano', 'Quantidade de Unidades', 'Quantidade de Unidades Condominiais',
       'Tamanho Médio da Unidade Condominial', 'Tamanho médio dos Terrenos',
       'Área Total dos terrenos-lotes', 'Área Total Ocupada',
       'Área Total Construída', 'Valor Total dos Terrenos',
       'Valor Total das Construções', 'CA médio', 'TO médio',
       'CA médio em lotes condominiais', 'TO médio em lotes condominiais',
       'CA médio em lotes não condominiais',
       'TO médio em lotes não condominiais', 'Comprimento Médio da Testada',
       'Número médio de Pavimentos', 'Fator de obsolecência médio',
       'Residencial', 'Comercial', 'Serviços', 'Industrial', 'Outros',
       'Percentual de Uso Residencial', 'Percentual de Uso Comercial',
       'Percentual de Uso Serviços', 'Percentual de Uso Industrial',
       'Percentual de Uso Outros'],
      dtype='object')

In [5]:
df_iptu

,ano,Quantidade de Unidades,Quantidade de Unidades Condominiais,Tamanho Médio da Unidade Condominial,Tamanho médio dos Terrenos,Área Total dos terrenos-lotes,Área Total Ocupada,Área Total Construída,Valor Total dos Terrenos,Valor Total das Construções,...,Residencial,Comercial,Serviços,Industrial,Outros,Percentual de Uso Residencial,Percentual de Uso Comercial,Percentual de Uso Serviços,Percentual de Uso Industrial,Percentual de Uso Outros
sqlc,,,,,,,,,,,,,,,,,,,,,
077001003600,1995,1,0,NaN,162.0,162,106,106,4860.0,14310.0,...,106,0,0,0,0,1.0,0.0,0.0,0.0,0.0
077001003600,1996,1,0,NaN,162.0,162,106,106,11016.0,18444.0,...,106,0,0,0,0,1.0,0.0,0.0,0.0,0.0
077001003600,1997,1,0,NaN,162.0,162,106,106,11988.0,20352.0,...,106,0,0,0,0,1.0,0.0,0.0,0.0,0.0
077001003600,1998,1,0,NaN,162.0,162,106,106,12798.0,21412.0,...,106,0,0,0,0,1.0,0.0,0.0,0.0,0.0
077001003600,1999,1,0,NaN,162.0,162,106,106,12960.0,21730.0,...,106,0,0,0,0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105126005000,2022,1,0,NaN,320.0,320,0,0,428800.0,0.0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
106101003900,2021,1,0,NaN,750.0,750,550,670,594750.0,529300.0,...,0,0,0,670,0,0.0,0.0,0.0,1.0,0.0
106101003900,2022,1,0,NaN,750.0,750,550,670,1003500.0,792610.0,...,0,0,0,670,0,0.0,0.0,0.0,1.0,0.0
